In [1]:
'''
Created on Jun 24, 2018

@author: Mark Holton
'''

import sys


import pandas as pd
import numpy as np


print("start")

start


Comments are for code below
Reads a text file in and converts it to a dataframe. Due to the large size of the file a smaller file is created with hard coded columns for testing.

In [2]:
#read the file in and
textfile = "./ag_map_with_alpha.txt"
originalDF=pd.read_csv(textfile,
                        sep = "\t")
masterDF = originalDF
masterDF_short=originalDF.loc[0:10,['#SampleID',  
                                            'age_years',    
                                            'antibiotic_history',    
                                            'bmi_cat',  
                                            'depression_bipolar_schizophrenia', 
                                            'mental_illness_type_depression', 
                                            'smoking_frequency']]


/home/markholton/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reads in user submited txt files and ask for user input to determine if data in the exclude file is to be filtered out or used to exclude everything that does not match it.

In [3]:
user_input_exclude = 'exclude' # input("Filename for file with column and data to exclude: ")
user_input_exclude_in_match = False # input('Is the "exclude" data to be excluded or kept (T for excluded and F for kept') == 'T'
user_input_control = 'control' # input("Filename for file with column and data to determine control: ")
user_input_experiment = 'experiment' # input("Filename for file with column and data to determine experimental: ")
user_input_match = 'match' # input("Filename for file with columns, type of value, and offset if needed to match samples: ")


user_input_exclude = './%s.txt'%(user_input_exclude)
user_input_control = './%s.txt'%(user_input_control)
user_input_experiment = './%s.txt'%(user_input_experiment )
user_input_match= './%s.txt'%(user_input_match)


print(user_input_exclude)
print(user_input_exclude_in_match)
print(user_input_control)
print(user_input_experiment)
print(user_input_match)

./exclude.txt
False
./control.txt
./experiment.txt
./match.txt


Turns the user files into dataframes

In [10]:
user_input_excludeDF = pd.read_csv(user_input_exclude, sep = "\t")
user_input_controlDF = pd.read_csv(user_input_control, sep = "\t")
user_input_experimentDF = pd.read_csv(user_input_experiment, sep = "\t")
user_input_matchDF = pd.read_csv(user_input_match, sep = "\t")



Filters out unwanted rows based on values in chosen columns

In [ ]:
masterDF = masterDF_short.copy()  
inputDF = user_input_excludeDF.copy()
exclude_if_match = user_input_exclude_in_match
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0

while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    print(column_name)
    masterDF = masterDF[
                masterDF[column_name].isin( inputDF[column_name].tolist() ) 
                == 
                ( not exclude_if_match ) ]
    c_counter = c_counter + 1

excludeDF = masterDF.copy()    
#masterDF

In [ ]:

#masterDF = masterDF.reset_index(drop=True)


Lables the samples based on if they are control or not

In [ ]:
masterDF = excludeDF.copy()

inputDF = user_input_controlDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0

while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    print(column_name)
    masterDF['control'] = masterDF[column_name].isin( inputDF[column_name].tolist() ).replace(True, 'control')   
    masterDF['control'] = masterDF['control'].replace(False, np.NaN )
    c_counter = c_counter + 1

controlDF = masterDF.copy()
controlDF

Lables the samples based on if they are experiment or not

In [ ]:
masterDF = controlDF.copy()

inputDF = user_input_experimentDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0

while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    print(column_name)
    masterDF['experiment'] = masterDF[column_name].isin( inputDF[column_name].tolist() )    
    c_counter = c_counter + 1

experDF = masterDF.copy()

Matches samples labled experiment to control samples

In [ ]:
masterDF = experDF.copy()

inputDF = user_input_matchDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0

while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    print(column_name)
    masterDF['match'] = masterDF[masterDF[column_name].isin( inputDF[column_name].tolist() ) ].    
    c_counter = c_counter + 1

matchDF = masterDF.copy()

In [ ]:
def Match( col_name, value_is_num, number_off=0):
    matchDF=masterDF
    matchDF['Matched to']='none'
    count_exper=0
    count_control=0
    while count_exper < matchDF.size:
        row = matchDF.loc[count_exper]
        if row.loc['control or experiment'] == 'exper':
            data_list=[]
            for col in col_name:
                col_value = row.loc(col)
                if(col_value == 'Unspecified'):
                    print("error some info not given for " + row.loc["#SampleID"])
                    count_exper = count_exper + 1
                    continue
                data_list.append(col_value)
                
            count_control=0
            while count_control < matchDF.size:
                row_match = matchDF.loc[count_control]
                if row_match.loc['control or experiment'] == 'control':
                    list_counter = 0
                    is_match = True
                    for col in col_name:
                        col_match_value = row_match.loc(col)
                        if(col_match_value == 'Unspecified'):
                            print("error some info not given for " + row_match.loc["#SampleID"])
                            count_control = count_control + 1
                            continue
                        if(value_is_num == 'False' 
                          and data_list.at(list_counter) != col_match_value ):
                            # match not found
                            is_match = False
                            count_control = count_control + 1
                            continue
                        if(value_is_num == 'True' 
                          and ( int(data_list.at(list_counter)) - number_off ) <= int(col_match_value)
                          and ( int(data_list.at(list_counter)) + number_off ) >= int(col_match_value) ):
                            # match not found
                            is_match = False
                            count_control = count_control + 1
                            continue
                        # if above cases not met then the row could still be a match
                        list_counter = list_counter + 1
                    
                    if is_match == True:
                        matchDF.loc[count_control, 'Matched to'] = row.loc['#SampleID']
                    count_control = count_control + 1
                
                count_exper = count_exper + 1

    return matchDF






In [ ]:
masterDF = copy.deepcopy(exper_controlDF)
while_count_three = while_count_two + 1
print(while_count_three)
masterDF['matched_to_samples'] = 'none'
#masterDF.assign(control_or_experiment = 'none')


while while_count_three < len(column_input_listcolumn_input_list) :
    col_name = column_input_list[while_count_three] 
    col_data_match = user_input_matchDF[col_name].tolist()
    val_type = col_data_match[0]
    num_off = 0
    if ( val_type == 'number' ):
        num_off = col_data_match[1]
    masterDF = Match( col_name, val_type, num_off):
    while_count_three = while_count_three + 1
print("done")
matchDF = copy.deepcopy(masterDF)
masterDF


